In [53]:
import numpy as np
from numpy.typing import NDArray
from typing import List

In [54]:
L = 9
def generate_grid(L: int) -> np.ndarray:
    return  np.random.choice([-1,1], size=(L,L))
state = generate_grid(9)

[[-1 -1  1 -1  1  1 -1 -1 -1]
 [-1  1 -1  1  1  1  1  1  1]
 [-1 -1 -1  1 -1 -1 -1  1  1]
 [-1  1 -1 -1 -1 -1  1 -1  1]
 [ 1 -1 -1 -1  1 -1  1  1  1]
 [ 1  1  1  1 -1 -1  1 -1 -1]
 [ 1  1 -1 -1 -1  1  1 -1  1]
 [ 1  1 -1  1  1  1 -1  1  1]
 [ 1 -1 -1 -1 -1  1 -1  1 -1]]


In [62]:
print(state)

[[-1 -1  1 -1  1  1 -1 -1 -1]
 [-1  1 -1  1  1  1  1  1  1]
 [-1 -1 -1  1 -1 -1 -1  1  1]
 [-1  1 -1 -1 -1 -1  1 -1  1]
 [ 1 -1 -1 -1  1 -1  1  1  1]
 [ 1  1  1  1 -1 -1  1 -1 -1]
 [ 1  1 -1 -1 -1  1  1 -1  1]
 [ 1  1 -1  1  1  1 -1  1  1]
 [ 1 -1 -1 -1 -1  1 -1  1 -1]]


In [55]:
def Energy(spins: np.ndarray) -> int:
    H = 0
    for row in spins:
        H += np.sum(row[:-1] * row[1:])
        #print(row[:-1] * row[1:])
        #print(H)
    
    for col in spins.T:
        H += np.sum(col[:-1] * col[1:])
        #print(col[:-1] * col[1:])
        #print(H)

    return H


In [56]:
print(Energy(state))

2


In [78]:
def DeltaE(initial_spins: np.ndarray, spin_to_flip: tuple[int, int]) -> int:
    #Calculates E_new - E_old
    L = initial_spins.shape[0]
    sum_of_neighbors = 0
    x,y = spin_to_flip[0], spin_to_flip[1]
    for dx in [-1,1]:
        if (0 <= x + dx < L):
             sum_of_neighbors += initial_spins[x + dx, y]

    for dy in [-1,1]:
        if (0 <= y + dy < L):
            sum_of_neighbors += initial_spins[x, y + dy]
     
    return -2 * sum_of_neighbors * initial_spins[x, y]


In [79]:
print(DeltaE(state, [4,4]))

8


In [75]:
new_state = state.copy()
new_state[4,4] = state[4,4] * -1

In [76]:
Energy(state)

2

In [77]:
Energy(new_state)

10

In [81]:
all_ones = np.ones((4,4))
print(all_ones)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [82]:
print(Energy(all_ones))

24.0


In [83]:
flip_all_ones = all_ones.copy()
flip_all_ones[1,0] = -1

In [84]:
print(Energy(flip_all_ones))

18.0


In [85]:
print(DeltaE(all_ones, [1,0]))

-6.0
